In [17]:
# Set up problem using TrajectoryOptimization.jl and RobotZoo.jl
using TrajectoryOptimization
using Altro
import RobotZoo.Cartpole
using StaticArrays, LinearAlgebra
using RobotDynamics

# Use the Cartpole model from RobotZoo
model = Cartpole()
print(typeof(dynamics(model, [0,0,0,0], [0,0])))
n,m = size(model)

# Define model discretization
N = 101
tf = 5.
dt = tf/(N-1)

# Define initial and final conditions
x0 = @SVector zeros(n)
xf = @SVector [0, pi, 0, 0]  # i.e. swing up

# Set up
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N)

# Add constraints
conSet = ConstraintList(n,m,N)
u_bnd = 3.0
bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
goal = GoalConstraint(xf)
add_constraint!(conSet, bnd, 1:N-1)
add_constraint!(conSet, goal, N)

# Initialization
u0 = @SVector fill(0.01,m)
U0 = [u0 for k = 1:N-1]

# Define problem
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet)
initial_controls!(prob, U0)

SVector{4, Float64}

In [10]:
# Solve with ALTRO
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)
altro = ALTROSolver(prob, opts)
print(catch_backtrace())
solve!(altro)

# Get some info on the solve
max_violation(altro)  # 3.42e-9
cost(altro)           # 1.55
iterations(altro)     # 40

# Extract the solution
X = states(altro)
U = controls(altro)

# Extract the solver statistics
stats = Altro.stats(altro)  # alternatively, solver.stats
stats.iterations             # 40, equivalent to iterations(solver)
stats.iterations_outer       # 4 (number of Augmented Lagrangian iterations)
stats.iterations_pn          # 1 (number of projected newton iterations)
stats.cost[end]              # terminal cost
stats.c_max[end]             # terminal constraint satisfaction
stats.gradient[end]          # terminal gradient of the Lagrangian
dstats = Dict(stats)         # get the per-iteration stats as a dictionary (can be converted to DataFrame)

Union{Ptr{Nothing}, Base.InterpreterIP}[]
SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 13
    Solve Time: 5.325215 (ms)

  Covergence
    Terminal Cost: 1.5370131414822565
    Terminal dJ: 0.4410171784000352
    Terminal gradient: 0.007145099487325638
    Terminal constraint violation: 4.157084010358858e-9
    Solve Status: SOLVE_SUCCEEDED


Dict{Symbol, Vector{T} where T} with 8 entries:
  :gradient        => [0.0538406, 0.0402182, 0.0601742, 0.0379797, 0.0551511, 0…
  :cost            => [1.54947, 1.54075, 1.53863, 1.54779, 1.60169, 2.17061, 7.…
  :iteration_pn    => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
  :iteration       => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
  :c_max           => [0.0895919, 0.0895919, 0.00922306, 0.00908885, 0.00901865…
  :penalty_max     => [1.0, 1.0, 10.0, 100.0, 1000.0, 10000.0, 10000.0, 10000.0…
  :iteration_outer => [1, 1, 2, 3, 4, 5, 5, 5, 6, 6, 6, 7, 7]
  :dJ              => [0.00308403, 0.0556398, 0.00211525, 0.000381931, 0.001031…